In [35]:
from massspecgym.data.datasets import MSnDataset, MSnRetrievalDataset, RetrievalDataset
import torch

In [36]:
file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_msn_library_pos_all_lib_MSn.mgf"
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/Retrieval/MassSpecGym_retrieval_candidates_mass.json"
split_file = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_split.tsv"

In [37]:
from massspecgym.data.transforms import MolFingerprinter, SpecTokenizer
from massspecgym.data import RetrievalDataset, MassSpecDataModule

In [38]:
pth_massspecgym_original = "/Users/macbook/CODE/Majer:MassSpecGym/data/MassSpecGym/MassSpecGym.tsv"

In [39]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_original = RetrievalDataset(
    pth=pth_massspecgym_original,
    candidates_pth=file_json,
    spec_transform=SpecTokenizer(n_peaks=n_peaks),
    mol_transform=MolFingerprinter(fp_size=fp_size),
)


In [40]:
# Init data module
data_module_original = MassSpecDataModule(
    dataset=dataset_original,
    batch_size=batch_size,
    num_workers=0,
)

In [41]:
data_module_original.prepare_data()
data_module_original.setup()

train_loader_original = data_module_original.train_dataloader()

Train dataset size: 194119
Val dataset size: 19429


In [42]:
tmp_original = []
for batch in train_loader_original:
    print(batch)
    tmp_original = batch
    break

I am not here
{'spec': tensor([[[2.7509e+02, 1.1000e+00],
         [2.0509e+02, 7.0070e-03],
         [2.2506e+02, 3.0030e-03],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[1.6910e+02, 1.1000e+00],
         [6.7551e+01, 7.8275e-03],
         [9.0745e+01, 8.8031e-03],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[4.6043e+02, 1.1000e+00],
         [5.1571e+01, 1.9918e-04],
         [5.4034e+01, 4.2192e-04],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        ...,

        [[3.4620e+02, 1.1000e+00],
         [4.4049e+01, 5.8350e-02],
         [5.7033e+01, 1.1010e-02],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[3.4722e+02, 1.1000e+00],
         [1.1506e+02, 2.

In [43]:
# Check the keys in the batch
print(tmp_original.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_original:
    print("batch_ptr is present:", tmp_original['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256, 256,  76, 256, 256, 256, 256, 256, 256, 256, 256, 256])


In [44]:
tmp_original.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [45]:
tmp_original['spec'].shape, tmp_original['mol'].shape, 

(torch.Size([12, 61, 2]), torch.Size([12, 4096]))

In [46]:
tmp_original['precursor_mz']

tensor([275.0914, 169.0970, 460.4260, 384.1470, 405.1600, 347.1700, 769.1700,
        369.1333, 551.3700, 346.2013, 347.2217, 215.0200])

In [47]:
tmp_original['adduct'], 

(['[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+Na]+',
  '[M+Na]+',
  '[M+H]+',
  '[M+Na]+',
  '[M+H]+',
  '[M+H]+',
  '[M+Na]+'],)

In [48]:
tmp_original['mol_freq']

tensor([  5.,  50.,   3.,   2.,  45., 405.,  17.,  20., 111.,   3.,  75.,  79.])

In [49]:
tmp_original['labels']

tensor([ True, False, False,  ..., False, False, False])

In [50]:
sum(tmp_original['labels'])

tensor(12)

In [51]:
torch.nonzero(tmp_original['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  512,  588,  844, 1100, 1356, 1612, 1868, 2124, 2380, 2636])

In [52]:
len(tmp_original['labels'])

2892

In [53]:
tmp_original['batch_ptr']

tensor([256, 256,  76, 256, 256, 256, 256, 256, 256, 256, 256, 256])

In [54]:
sum(tmp_original['batch_ptr'])

tensor(2892)

In [55]:
len(tmp_original['candidates'])

2892

In [56]:
tmp_original['candidates'][5].shape

torch.Size([4096])

In [57]:
len(tmp_original['candidates_smiles'])

2892

# MSnRetrieval

In [5]:
from massspecgym.featurize import SpectrumFeaturizer

In [6]:
config = {
    'features': ['collision_energy', 'ionmode', 'adduct', 'spectrum_stats', 'atom_counts', 'value', "retention_time", 'ion_source', 'binned_peaks'],
    'feature_attributes': {
        'atom_counts': {
            'top_n_atoms': 12,
            'include_other': True,
        },
    },
}

In [7]:
featurizer = SpectrumFeaturizer(config, mode='torch')

In [8]:
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/candidates_generation/MassSpecGymMSn_retrieval_candidates_formula.json"

In [9]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_msn = MSnRetrievalDataset(
    pth=file_mgf,
    candidates_pth=file_json,
    featurizer=featurizer,
    mol_transform=MolFingerprinter(fp_size=fp_size),
    max_allowed_deviation=0.005
)


Total valid indices: 16476
Dataset length: 16476


In [10]:
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
def is_valid_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return False
    Chem.SanitizeMol(mol)
    return True

In [11]:
# Init data module
data_module_msn = MassSpecDataModule(
    dataset=dataset_msn,
    batch_size=batch_size,
    split_pth=split_file,
    num_workers=0,
)

In [13]:
data_module_msn.prepare_data()
data_module_msn.setup()

train_loader_msn = data_module_msn.train_dataloader()

Train dataset size: 12536
Val dataset size: 1952


In [14]:
tmp_msn = []
for batch in train_loader_msn:
    print(batch)
    tmp_msn = batch
    break

{'spec': DataBatch(x=[124, 1039], edge_index=[2, 224], batch=[124], ptr=[13]), 'mol': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'precursor_mz': tensor([362.1322, 705.2025, 444.2143, 401.1919, 379.2479, 477.2384, 328.1305,
        270.1489, 378.1747, 558.2300, 397.1329, 639.3586]), 'adduct': ['[M+H]+', '[M+H]+', '[M+H]+', '[M+H-2H2O]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H-H2O]+', '[M+H]+', '[M+H]+', '[M+H]+', '[M+H]+'], 'identifier': ['0040250_0000000', '0006256_0000000', '0060488_0000000', '0009601_0000000', '0009532_0000000', '0086220_0000000', '0074197_0000000', '0016939_0000000', '0064184_0000000', '0089115_0000000', '0028740_0000000', '0009102_0000000'], 'mol_freq': tensor([1., 1., 1., 4., 2., 1., 1., 4., 1., 2., 1., 3.]), 'smiles': ['CC1=NC(=NC=C1)SCC(=O)N2C3=CC=CC=C3CCC4

In [15]:
# Check the keys in the batch
print(tmp_msn.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_msn:
    print("batch_ptr is present:", tmp_msn['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256,   7, 256,  69, 256,  24, 256, 256, 256,  63, 256,  17])


#### Shold be dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [16]:
tmp_msn.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'identifier', 'mol_freq', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [17]:
tmp_msn['spec'] 

DataBatch(x=[124, 1039], edge_index=[2, 224], batch=[124], ptr=[13])

In [18]:
tmp_msn['precursor_mz']

tensor([362.1322, 705.2025, 444.2143, 401.1919, 379.2479, 477.2384, 328.1305,
        270.1489, 378.1747, 558.2300, 397.1329, 639.3586])

In [19]:
tmp_msn['mol_freq']

tensor([1., 1., 1., 4., 2., 1., 1., 4., 1., 2., 1., 3.])

In [20]:
tmp_msn['identifier']

['0040250_0000000',
 '0006256_0000000',
 '0060488_0000000',
 '0009601_0000000',
 '0009532_0000000',
 '0086220_0000000',
 '0074197_0000000',
 '0016939_0000000',
 '0064184_0000000',
 '0089115_0000000',
 '0028740_0000000',
 '0009102_0000000']

In [21]:
tmp_msn['adduct']

['[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H-2H2O]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H-H2O]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+',
 '[M+H]+']

In [22]:
tmp_msn['mol'].shape

torch.Size([12, 4096])

In [23]:
tmp_msn['smiles']

['CC1=NC(=NC=C1)SCC(=O)N2C3=CC=CC=C3CCC4=CC=CC=C42',
 'CC(=O)OC1COC(C(C1OC(=O)C)OC(=O)C)C2=C(C3=C(C=C2O)OC(=CC3=O)C4=CC=C(C=C4)OC5C(C(C(C(O5)CO)O)O)O)OC',
 'CC1=C(C(=NO1)C)C2=C(C=C3C(=C2)NC4=C3C(=NC(=N4)C)NC5=CC(=NN5C)C6CC6)OC',
 'CC(C)C(C(=O)OCC(C(C(C(COC(=O)C(C(C)C)NC=O)O)O)O)O)NC=O',
 'CC1CCC2(C(C1(C)CCC(C)CC(=O)O)CC(=O)C=C2COC(=O)C)C',
 'C[C@@H]1CC2=C(C=CC3=C2C=NN3)[C@H](N1CC(F)(F)F)C4=NC=C(C=C4)NC5CN(C5)CCCF',
 'C1=CC=C(C=C1)C2=NN3C=CC=CC3=C2C4=NN=C5C(=C4)C(=NN5)N',
 'CN1CC[C@@]23C=C[C@@H](C[C@@H]2OC4=C(C=CC(=C34)C1)OC)O',
 'CCC(CC)C(C1=CC=C(C=C1)NC2=NC3=CC=CC=C3S2)N4C=NC=N4',
 'CC1=CC(=C(C=C1C2CCNCC2)OC(C)C)NC3=NC=C(C(=N3)NC4=CC=CC=C4S(=O)(=O)C(C)C)Cl',
 'CC1=NN(C(=C1)NC(=O)C2CCCC2)C3=NC(=CS3)C4=CC5=C(C=C4)OCO5',
 'CCC(C)C(C(=O)OC(C(C)CC)C(=O)OC(C(C)CC)C(=O)OC(C(C)CC)C(=O)OCC(C(C(C(CO)O)O)O)O)O']

In [24]:
tmp_msn['labels']

tensor([ True, False, False,  ..., False, False, False])

In [25]:
sum(tmp_msn['labels'])

tensor(12)

In [26]:
torch.nonzero(tmp_msn['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  263,  519,  588,  844,  868, 1124, 1380, 1636, 1699, 1955])

In [27]:
len(tmp_msn['labels'])

1972

In [28]:
tmp_msn['batch_ptr']

tensor([256,   7, 256,  69, 256,  24, 256, 256, 256,  63, 256,  17])

In [29]:
sum(tmp_msn['batch_ptr'])

tensor(1972)

In [30]:
len(tmp_msn['candidates'])

1972

In [31]:
tmp_msn['candidates'][5].shape

torch.Size([4096])

### Compare Shapes

In [58]:
tmp_msn.keys() == tmp_original.keys()

True

In [59]:
tmp_msn['batch_ptr'].shape == tmp_original['batch_ptr'].shape

True

In [60]:
tmp_msn['batch_ptr'].dtype == tmp_original['batch_ptr'].dtype

True

In [61]:
sum(tmp_msn['batch_ptr']), len(tmp_msn['candidates_smiles'])

(tensor(1972), 1972)

In [62]:
sum(tmp_original['batch_ptr']), len(tmp_original['candidates_smiles'])

(tensor(2892), 2892)

In [63]:
tmp_msn['labels'].dtype == tmp_original['labels'].dtype

True

In [64]:
tmp_msn['candidates'].dtype == tmp_original['candidates'].dtype

True

In [65]:
tmp_msn['candidates'][0].shape == tmp_original['candidates'][0].shape

True

In [66]:
tmp_msn['candidates'][5].shape == tmp_original['candidates'][5].shape

True

In [67]:
type(tmp_msn['smiles']) == type(tmp_original['smiles'])

True

In [68]:
type(tmp_msn['smiles'][0]) == type(tmp_original['smiles'][0])

True

In [69]:
len(tmp_msn['smiles']) == len(tmp_original['smiles'])

True

In [70]:
type(tmp_msn['candidates_smiles']) == type(tmp_original['candidates_smiles'])

True

In [71]:
type(tmp_msn['candidates_smiles'][0]) == type(tmp_original['candidates_smiles'][0])

True

In [72]:
len(tmp_msn['candidates_smiles'])

1972